In [ ]:
# jupyter nbconvert tests/bigbrotr_data_overview.ipynb --to html/pdf --no-input

# Bigbrotr Database Data Overview

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import sys
sys.path.append("../src")

import seaborn as sns
sns.set_theme(style="whitegrid")

import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (12, 6)

import os
from bigbrotr import Bigbrotr
from event import Event
from relay import Relay
from relay_metadata import RelayMetadata
import utils
import pandas as pd
from datetime import datetime
from matplotlib_venn import venn3
import matplotlib.patches as mpatches
import numpy as np
import psycopg2

In [ ]:
DB_HOST = 'localhost'
DB_PORT =5432
DB_USER = 'admin'
DB_PASSWORD = 'admin'
DB_NAME = 'bigbrotr'

bigbrotr = Bigbrotr(
    host=DB_HOST,
    port=DB_PORT,
    user=DB_USER,
    password=DB_PASSWORD,
    dbname=DB_NAME
)

bigbrotr.connect()

## Database Schema

In [ ]:
def get_schema_overview(conn):
    q = '''
    SELECT table_name, column_name, data_type
    FROM information_schema.columns
    WHERE table_schema = 'public'
    ORDER BY table_name, ordinal_position;
    '''
    return pd.read_sql(q, conn)

schema_df = get_schema_overview(bigbrotr.conn)
for table_name in schema_df['table_name'].unique():
    df = schema_df[schema_df['table_name'] == table_name].drop(columns='table_name')
    display(table_name.capitalize())
    print(df.to_markdown(index=False))
    print("\n")

## Row Counts per Table


In [ ]:
# def get_row_counts(conn):
#     q = '''
#     SELECT relname as table, n_live_tup as row_count
#     FROM pg_stat_user_tables
#     ORDER BY n_live_tup DESC;
#     '''
#     return pd.read_sql(q, conn)

# row_counts = get_row_counts(bigbrotr.conn)
# display(row_counts)

In [ ]:
def bytes_to_gb(b):
    return b / (1024 ** 3)

conn = psycopg2.connect(
    dbname="bigbrotr",
    user="admin",
    password="admin",
    host="localhost",
    port=5432
)
cursor = conn.cursor()

# Recupera tutte le tabelle nel tuo schema pubblico
cursor.execute("""
    SELECT table_name 
    FROM information_schema.tables 
    WHERE table_schema = 'public' AND table_type = 'BASE TABLE'
""")
tables = [row[0] for row in cursor.fetchall()]

print("📊 Analisi dello spazio per tabella:\n")

for table in tables:
    print(f"🧾 TABELLA: {table}")
    
    # Numero righe
    cursor.execute(f"SELECT COUNT(*) FROM {table}")
    total_rows = cursor.fetchone()[0]
    print(f"Numero righe: {total_rows}")

    if total_rows == 0:
        print(" (Tabella vuota)\n")
        continue

    # Calcola dimensione media per colonna
    cursor.execute(f"""
        SELECT string_agg(
            'AVG(pg_column_size(\"' || column_name || '\")) AS \"' || column_name || '\"',
            ', '
        )
        FROM information_schema.columns
        WHERE table_name = %s AND table_schema = 'public'
    """, (table,))
    agg_expr = cursor.fetchone()[0]

    avg_size_query = f"SELECT {agg_expr} FROM {table}"
    cursor.execute(avg_size_query)
    avg_sizes = cursor.fetchone()
    colnames = [desc.name for desc in cursor.description]

    print("{:<20} {:>12} {:>12}".format("Colonna", "Avg (bytes)", "Totale (GB)"))
    total_data_bytes = 0

    for i, col in enumerate(colnames):
        avg_bytes = float(avg_sizes[i])
        total_bytes = avg_bytes * total_rows
        total_data_bytes += total_bytes
        print("{:<20} {:>12.2f} {:>12.2f}".format(col, avg_bytes, bytes_to_gb(total_bytes)))

    print(f"Totale dati stimati: {bytes_to_gb(total_data_bytes):.2f} GB")

    # Recupera indici della tabella con dimensione
    cursor.execute(f"""
        SELECT 
            indexname, 
            pg_relation_size(indexrelid) AS size_bytes
        FROM pg_indexes
        JOIN pg_class ON pg_class.relname = indexname
        JOIN pg_index ON pg_class.oid = pg_index.indexrelid
        WHERE tablename = %s
    """, (table,))
    index_data = cursor.fetchall()

    total_index_bytes = 0
    print("\n📦 Indici:")
    print("{:<30} {:>12}".format("Indice", "Size (GB)"))
    for index_name, size_bytes in index_data:
        total_index_bytes += size_bytes
        print("{:<30} {:>12.2f}".format(index_name, bytes_to_gb(size_bytes)))

    print(f"Totale indici: {bytes_to_gb(total_index_bytes):.2f} GB")
    print(f"Totale complessivo stimato: {bytes_to_gb(total_data_bytes + total_index_bytes):.2f} GB\n")
    print("─" * 60)

cursor.close()
conn.close()

## Relays Overview

In [ ]:
relays_w_metadata = pd.read_sql(
    '''
    SELECT DISTINCT ON (relay_url) relay_url
    FROM relay_metadata
    ''', bigbrotr.conn
)
relays = pd.read_sql('SELECT url, network FROM relays', bigbrotr.conn)
relays['metadata'] = relays['url'].isin(relays_w_metadata['relay_url'])
with_metadata_count = relays['metadata'].sum()
total_relays = relays.shape[0]
print(f"Total relays: {total_relays}, with metadata: {with_metadata_count} ({with_metadata_count/total_relays:.2%})")
print("Network distribution for relays with metadata:")
tmp = relays[relays['metadata']]['network'].value_counts().reset_index()
tmp['perc'] = tmp['count'] / tmp['count'].sum() * 100
for _, row in tmp.iterrows():
    print(f"- {row['network']}: {row['count']} ({row['perc']:.2f}%)")

In [ ]:
relay_metadata = pd.read_sql(
    '''
    SELECT *
    FROM relay_metadata
    ''', bigbrotr.conn
)
relay_metadata = pd.merge(relay_metadata, relays.filter(['url', 'network']).rename(columns={'url': 'relay_url'}), how='left')

In [ ]:
relay_metadata.value_counts('relay_url').plot(kind='hist', title='Relay metadata count distribution')

In [ ]:
tmp = relay_metadata.copy()
tmp['date'] = pd.to_datetime(tmp['generated_at'], unit='s').dt.date
tmp_daily = tmp.sort_values(['relay_url', 'generated_at']).drop_duplicates(subset=['relay_url', 'date'])
tmp_daily = tmp_daily.set_index(['relay_url', 'date'])

In [ ]:
print(f"Total unique relays with metadata: {tmp_daily.index.get_level_values('relay_url').nunique()}")

# --- Prep: Assume tmp_daily already exists ---
# Count number of relays per day
daily_counts = tmp_daily.reset_index().groupby('date')['relay_url'].nunique()
daily_counts = daily_counts.to_frame(name='relay_count')
daily_counts.index = pd.to_datetime(daily_counts.index)

# Total number of unique relays (to use as cap)
max_relays = tmp_daily.index.get_level_values('relay_url').nunique()

# Add week and weekday columns
daily_counts['week'] = daily_counts.index.isocalendar().week
daily_counts['dow'] = daily_counts.index.weekday  # Monday=0

# Pivot for heatmap
pivot_table = daily_counts.pivot(index='week', columns='dow', values='relay_count')

# Cap values at total number of relays
pivot_table = pivot_table.clip(upper=max_relays)

# Optional: Replace weekday numbers with labels
pivot_table.columns = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']

# Plot heatmap with annotations
plt.figure(figsize=(12, 6))
sns.heatmap(pivot_table, cmap='YlGnBu', annot=True, fmt='.0f', linewidths=0.5, linecolor='gray',
            vmin=0, vmax=max_relays, cbar_kws={'label': 'Relay Count'})
plt.title('Number of Relay URLs Monitored Per Day')
plt.xlabel('Day of Week')
plt.ylabel('Week Number')
plt.tight_layout()
plt.show()

In [ ]:
metrics = ['connection_success', 'nip11_success', 'readable', 'writable', 'openable']
networks = ['clearnet', 'tor', 'all']
nip11_cols = ['name', 'description', 'banner', 'icon', 'pubkey', 'contact', 'supported_nips', 'software', 'version', 'privacy_policy', 'terms_of_service', 'limitation', 'extra_fields']

def mode_of_series(series):
    """Calculate the mode (most frequent value) of a Series."""
    if series.empty:
        return None
    mode_series = series.mode()
    if mode_series.empty:
        return None
    return mode_series.iloc[0]

def mode_of_lists(series):
    tuples = series.apply(lambda x: tuple(x) if isinstance(x, list) else x)
    if tuples.empty:
        return []
    mode_series = tuples.mode()
    if mode_series.empty:
        return []
    mode_tuple = mode_series.iloc[0]
    return list(mode_tuple) if isinstance(mode_tuple, tuple) else mode_tuple

def mode_of_dicts(series):
    def dict_to_tuple(d):
        if isinstance(d, dict):
            return tuple(sorted(d.items()))
        return d
    tuples = series.apply(dict_to_tuple)
    if tuples.empty:
        return {}
    mode_series = tuples.mode()
    if mode_series.empty:
        return {}
    mode_tuple = mode_series.iloc[0]
    return dict(mode_tuple) if isinstance(mode_tuple, tuple) else mode_tuple

In [ ]:
def plot_mean_rtt_percentile_filter(df, rtt_columns, lower_pct=0.05, upper_pct=0.95):
    """
    df: dataframe originale con 'network' e colonne RTT
    rtt_columns: lista colonne RTT su cui filtrare outlier con percentili
    lower_pct: percentile inferiore (es. 0.05 = 5%)
    upper_pct: percentile superiore (es. 0.95 = 95%)
    """
    df_clean = df.dropna(subset=rtt_columns).copy()
    
    for col in rtt_columns:
        low_val = df_clean[col].quantile(lower_pct)
        high_val = df_clean[col].quantile(upper_pct)
        df_clean = df_clean[(df_clean[col] >= low_val) & (df_clean[col] <= high_val)]
    
    mean_rtt = df_clean.groupby('network')[rtt_columns].mean()
    best_rtt = mean_rtt.min()
    percent_diff = ((mean_rtt - best_rtt) / best_rtt * 100).round(1)
    
    ax = mean_rtt.plot(kind='bar', figsize=(12, 6))
    plt.title(f'Mean RTT by Network (Filtered between {int(lower_pct*100)}% and {int(upper_pct*100)}% percentiles)')
    plt.ylabel('RTT (ms)')
    plt.xlabel('Network')
    plt.grid(True)
    plt.xticks(rotation=0)
    plt.tight_layout()
    
    # Etichette % sopra barre
    for i, col in enumerate(mean_rtt.columns):
        for j, val in enumerate(mean_rtt[col]):
            pct = percent_diff.iloc[j, i]
            ax.text(j + i*0.17 - 0.17, val + 1, f'+{pct}%', ha='center', va='bottom', fontsize=8)
    
    plt.show()

# Uso:
df_tmp = tmp_daily.reset_index()
plot_mean_rtt_percentile_filter(df_tmp, ['rtt_open', 'rtt_read', 'rtt_write'], lower_pct=0.05, upper_pct=0.95)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def plot_rtt_cdfs_by_network(df, rtt_columns, lower_pct=0.05, upper_pct=0.95):
    """
    df: dataframe con colonna 'network', colonne RTT e multi-indice (relay_id, data_misurazione)
    rtt_columns: lista colonne RTT (es. ['rtt_open', 'rtt_read', 'rtt_write'])
    lower_pct: percentile inferiore per outlier removal
    upper_pct: percentile superiore per outlier removal
    """
    df_clean = df.dropna(subset=rtt_columns).copy()

    # Filtro outlier globalmente
    for col in rtt_columns:
        low = df_clean[col].quantile(lower_pct)
        high = df_clean[col].quantile(upper_pct)
        df_clean = df_clean[(df_clean[col] >= low) & (df_clean[col] <= high)]

    n_networks = len(networks)

    fig, axes = plt.subplots(n_networks, 1, figsize=(10, 5 * n_networks), sharex=True)

    if n_networks == 1:
        axes = [axes]  # Se c'è solo un network

    for ax, net in zip(axes, networks):
        if net == 'all':
            df_net = df_clean
        else:
            df_net = df_clean[df_clean['network'] == net]
        for col in rtt_columns:
            sorted_vals = np.sort(df_net[col])
            cdf = np.linspace(0, 1, len(sorted_vals))
            ax.plot(sorted_vals, cdf, label=col)
        ax.set_title(f'CDF RTT - Network: {net}')
        ax.set_ylabel('CDF')
        ax.grid(True)
        ax.legend()

    axes[-1].set_xlabel('RTT (ms)')
    plt.tight_layout()
    plt.show()

df_tmp = tmp_daily.reset_index()  # Assicurati che 'network' sia colonna, non indice
plot_rtt_cdfs_by_network(df_tmp, ['rtt_open', 'rtt_read', 'rtt_write'], lower_pct=0.05, upper_pct=0.95)

In [ ]:
# Prepara i dati heatmap in un dizionario (simula il calcolo precedente)
heatmaps_data = {}

for network in networks:
    if network == 'all':
        tmp_network = tmp_daily
    else:
        tmp_network = tmp_daily[tmp_daily['network'] == network]
    mode_df = tmp_network.groupby(level=0)[metrics].agg(mode_of_series)

    mode_counts = pd.DataFrame({
        col: mode_df[col].value_counts() for col in metrics
    }).fillna(0).astype(int)

    mode_percent = mode_counts.div(mode_counts.sum(axis=0), axis=1) * 100

    heatmaps_data[network] = mode_percent.T

# Plot
fig, axs = plt.subplots(1, 3, figsize=(18, 6))
fig.suptitle("Percentage of Relays per Metric", fontsize=16, y=1.05)

for i, network in enumerate(networks):
    sns.heatmap(
        heatmaps_data[network],
        annot=True,
        fmt=".1f",
        cmap="YlGnBu",
        cbar=(i == 2),
        ax=axs[i],
        cbar_kws={'label': 'Percentage (%)'}
    )
    axs[i].set_title(f"Network: {network}", fontsize=12)
    axs[i].set_xlabel("Value")
    axs[i].set_ylabel("Metric")
    axs[i].tick_params(axis='x', rotation=0)

plt.tight_layout()
plt.subplots_adjust(top=0.85)
plt.show()

In [ ]:
mode_df = tmp_daily.groupby(level=0)[metrics+['network']].agg(mode_of_series)
for network in networks:
    if network == 'all':
        tmp_network = mode_df
    else:
        tmp_network = mode_df[mode_df['network'] == network]
    tmp_network = tmp_network.drop(columns='network')
    print(f"\nNetwork: {network}")
    display(tmp_network.value_counts().reset_index(name='count').rename(columns={'index': 'value'}).sort_values(by='count', ascending=False))

In [ ]:
subset_labels = {
    '100': 'Openable only',
    '010': 'Readable only',
    '001': 'Writable only',
    '110': 'Openable & Readable',
    '101': 'Openable & Writable',
    '011': 'Readable & Writable',
    '111': 'Openable & Readable & Writable',
}

venn_sets = {}

# Prepara i set per ogni network
for network in networks:
    if network == 'all':
        tmp_network = tmp_daily
    else:
        tmp_network = tmp_daily[tmp_daily['network'] == network]
    mode_df = tmp_network.groupby(level=0)[['openable', 'readable', 'writable']].agg(mode_of_series)

    set_openable = set(mode_df[mode_df['openable'] == True].index)
    set_readable = set(mode_df[mode_df['readable'] == True].index)
    set_writable = set(mode_df[mode_df['writable'] == True].index)

    venn_sets[network] = (set_openable, set_readable, set_writable)

# Plot
fig, axs = plt.subplots(1, 3, figsize=(18, 6))
fig.suptitle("Relay Status Combinations", fontsize=16, y=1.05)

for i, network in enumerate(networks):
    set_openable, set_readable, set_writable = venn_sets[network]
    v = venn3([set_openable, set_readable, set_writable], ax=axs[i])

    # Nascondi nomi e numeri sui cerchi
    for text in v.set_labels:
        if text:
            text.set_visible(False)
    for subset_id in subset_labels.keys():
        label = v.get_label_by_id(subset_id)
        if label:
            label.set_text('')

    total_relays = len(set_openable.union(set_readable).union(set_writable))
    sets = [set_openable, set_readable, set_writable]

    patches = []
    for subset_id, label_text in subset_labels.items():
        patch = v.get_patch_by_id(subset_id)
        if patch:
            included_sets = [subset_id[j] == '1' for j in range(3)]

            selected = sets[0] if included_sets[0] else set()
            for j in range(1, 3):
                if included_sets[j]:
                    if selected:
                        selected = selected.intersection(sets[j])
                    else:
                        selected = sets[j]
            for j in range(3):
                if not included_sets[j]:
                    selected = selected.difference(sets[j])

            count = len(selected)
            percent = (count / total_relays * 100) if total_relays > 0 else 0
            label_with_count = f"{label_text}: {count} ({percent:.1f}%)"
            patches.append(mpatches.Patch(color=patch.get_facecolor(), label=label_with_count))

    axs[i].legend(handles=patches, loc='upper right', fontsize=8)
    axs[i].set_title(f"Network: {network}", fontsize=12)

plt.tight_layout()
plt.subplots_adjust(top=0.85)
plt.show()

In [ ]:
# Collect results for each relay_url
results = []

for relay_url, group in tmp_daily.groupby(level='relay_url'):
    row = {'relay_url': relay_url}
    for col in nip11_cols:
        if col not in group.columns:
            row[col] = None
            continue

        if col == 'supported_nips':
            row[col] = mode_of_lists(group[col])
        elif col in ['limitation', 'extra_fields']:
            row[col] = mode_of_dicts(group[col])
        else:
            row[col] = mode_of_series(group[col])
    row['network'] = group['network'].iloc[0]
    results.append(row)

# Convert to DataFrame
nip11_modes_df = pd.DataFrame(results).set_index('relay_url')
nip11_modes_df['supported_nips'] = nip11_modes_df['supported_nips'].apply(lambda x: np.nan if x == [] else x)
nip11_modes_df['limitation'] = nip11_modes_df['limitation'].apply(lambda x: np.nan if x == {} else x)
nip11_modes_df['extra_fields'] = nip11_modes_df['extra_fields'].apply(lambda x: np.nan if x == {} else x)

In [ ]:
# Prepara un DataFrame che conterrà tutte le percentuali per network e colonne nip11
heatmap_data = pd.DataFrame(index=nip11_cols)

for network in networks:
    if network == 'all':
        tmp_network = nip11_modes_df
    else:
        tmp_network = nip11_modes_df[nip11_modes_df['network'] == network]

    percentages = []
    total_relays = tmp_network.shape[0]
    for col in nip11_cols:
        if col not in tmp_network.columns or total_relays == 0:
            perc = 0.0
        else:
            perc = (tmp_network[col].notna().sum() / total_relays) * 100
        percentages.append(perc)
    heatmap_data[network] = percentages

# Calcola il divario assoluto tra clearnet e tor
if 'clearnet' in heatmap_data.columns and 'tor' in heatmap_data.columns:
    heatmap_data['divario'] = (heatmap_data['clearnet'] - heatmap_data['tor']).abs()
    heatmap_data = heatmap_data.sort_values(by='divario', ascending=False)
    heatmap_data = heatmap_data.drop(columns='divario')

# Plot
plt.figure(figsize=(12, 6))
sns.heatmap(
    heatmap_data,
    annot=True,
    fmt=".1f",
    cmap="YlGnBu",  # più indicato per divergenze
    cbar_kws={'label': 'Percentage (%)'},
    linewidths=0.5,
    linecolor='gray'
)

plt.title("Percentage of Relays per NIP-11 Column and Network (Sorted by Clearnet-Tor Gap)")
plt.ylabel("NIP-11 Column")
plt.xlabel("Network")
plt.xticks(rotation=0, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()


In [ ]:
def calc_heatmap_data_for_dict_column(df, networks, col_name, sort_by_gap=True):
    all_keys = set()
    for _, row in df.iterrows():
        d = row[col_name]
        if isinstance(d, dict):
            all_keys.update(d.keys())
    all_keys = sorted(all_keys)

    heatmap_data = pd.DataFrame(index=all_keys)

    for network in networks:
        if network == 'all':
            tmp_network = df
        else:
            tmp_network = df[df['network'] == network]

        total_relays = tmp_network.shape[0]
        percentages = []
        for key in all_keys:
            if total_relays == 0:
                perc = 0.0
            else:
                count = tmp_network[col_name].apply(lambda d: isinstance(d, dict) and key in d).sum()
                perc = (count / total_relays) * 100
            percentages.append(perc)
        heatmap_data[network] = percentages

    if sort_by_gap and 'clearnet' in heatmap_data.columns and 'tor' in heatmap_data.columns:
        heatmap_data['gap'] = (heatmap_data['clearnet'] - heatmap_data['tor']).abs()
        heatmap_data = heatmap_data.sort_values('gap', ascending=False).drop(columns='gap')
    else:
        heatmap_data['total'] = heatmap_data.sum(axis=1)
        heatmap_data = heatmap_data.sort_values('total', ascending=False).drop(columns='total')

    return heatmap_data


# Calcola heatmap per limitation ordinata
heatmap_limitation = calc_heatmap_data_for_dict_column(nip11_modes_df, networks, 'limitation')

# Calcola heatmap per extra_fields ordinata
heatmap_extra_fields = calc_heatmap_data_for_dict_column(nip11_modes_df, networks, 'extra_fields')


# Plot heatmap limitation
plt.figure(figsize=(12, max(6, len(heatmap_limitation)*0.4)))
sns.heatmap(
    heatmap_limitation,
    annot=True,
    fmt=".1f",
    cmap="YlGnBu",
    cbar_kws={'label': 'Percentage (%)'},
    linewidths=0.5,
    linecolor='gray'
)
plt.title("Percentage of Relays per Limitation Type and Network (Ordered by Clearnet-Tor Gap)")
plt.ylabel("Limitation Type")
plt.xlabel("Network")
plt.xticks(rotation=0, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()


# Plot heatmap extra_fields
plt.figure(figsize=(12, max(6, len(heatmap_extra_fields)*0.4)))
sns.heatmap(
    heatmap_extra_fields,
    annot=True,
    fmt=".1f",
    cmap="YlGnBu",
    cbar_kws={'label': 'Percentage (%)'},
    linewidths=0.5,
    linecolor='gray'
)
plt.title("Percentage of Relays per Extra Field and Network (Ordered by Clearnet-Tor Gap)")
plt.ylabel("Extra Field")
plt.xlabel("Network")
plt.xticks(rotation=0, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()


In [ ]:
mode_relays = pd.merge(
    tmp_daily.groupby(level=0)[metrics].agg(mode_of_series).reset_index(),
    nip11_modes_df.reset_index()
)

In [ ]:
# Prepara i dati
tmp_mode = mode_relays.filter(['relay_url', 'network', 'writable', 'readable', 'limitation']).copy()
tmp_mode['restricted_writes'] = tmp_mode['limitation'].apply(lambda x: x.get('restricted_writes', None) if isinstance(x, dict) else False)
tmp_mode['payment_required'] = tmp_mode['limitation'].apply(lambda x: x.get('payment_required', None) if isinstance(x, dict) else False)
tmp_mode = tmp_mode.drop(columns='limitation')
tmp_mode = tmp_mode.set_index(['relay_url', 'network'])

# print number of relays
print(f"Number of relays in cleanet and tor: {tmp_mode.shape[0]}")
print(f"Number of relays in cleanet: {tmp_mode.xs('clearnet', level='network').shape[0]}")
print(f"Number of relays in tor: {tmp_mode.xs('tor', level='network').shape[0]}")

# Numero di heatmap da plottare
n = len(networks)
fig, axes = plt.subplots(1, n, figsize=(7 * n, 7))  # figura più larga e alta

# Se c'è un solo network, axes non è una lista → forzalo
if n == 1:
    axes = [axes]

for ax, network in zip(axes, networks):
    if network == 'all':
        group = tmp_mode.copy()
    else:
        group = tmp_mode.xs(network, level='network')

    corr_matrix = group.corr()

    # Calcolo della matrice con numero di coppie valide
    valid_counts = pd.DataFrame(
        np.zeros_like(corr_matrix, dtype=int),
        index=corr_matrix.index,
        columns=corr_matrix.columns
    )

    for col1 in group.columns:
        for col2 in group.columns:
            valid_counts.loc[col1, col2] = group[[col1, col2]].dropna().shape[0]

    # Prepara annotazioni combinate: "r=0.75\n(n=123)"
    annot_matrix = corr_matrix.copy()
    for i in corr_matrix.index:
        for j in corr_matrix.columns:
            r = corr_matrix.loc[i, j]
            n = valid_counts.loc[i, j]
            annot_matrix.loc[i, j] = f"{r:.2f}\n(n={n})"

    sns.heatmap(
        corr_matrix,
        annot=annot_matrix,
        fmt="",  # fmt vuoto perché l'annotazione è già formattata
        cmap="coolwarm",
        center=0,
        linewidths=0.7,
        linecolor='gray',
        annot_kws={"size": 11},
        cbar_kws={'shrink': 0.8},
        ax=ax
    )
    ax.set_title(f"{network}", fontsize=16)
    ax.tick_params(axis='x', rotation=90, labelsize=12)
    ax.tick_params(axis='y', rotation=0, labelsize=12)

plt.tight_layout(rect=[0, 0, 1, 0.92])  # spazio per il titolo
plt.subplots_adjust(wspace=0.35)  # più spazio fra i plot
plt.suptitle("Correlation Matrices of Relay Properties by Network", fontsize=20)
plt.show()

In [ ]:
for network in networks:
    if network == 'all':
        group = tmp_mode.copy()
    else:
        group = tmp_mode.xs(network, level='network')

    print(f"\nNetwork: {network}")
    display(group.value_counts().rename('count').reset_index().sort_values(by='count', ascending=False))

In [ ]:
if 'relay_synchronization.csv' not in os.listdir('.'):
    query = """
    SELECT
        latest.relay_url AS relay_url,
        e.created_at AS timestamp,
        latest.seen_at AS seen_at
    FROM (
        SELECT DISTINCT ON (relay_url)
            relay_url,
            seen_at,
            event_id
        FROM
            events_relays
        ORDER BY
            relay_url,
            seen_at DESC
    ) AS latest
    JOIN events e ON e.id = latest.event_id;
    """
    bigbrotr.execute(query)
    rows = bigbrotr.fetchall()
    df = pd.DataFrame(rows, columns=['relay_url', 'timestamp', 'seen_at'])
    df['timestamp_month'] = pd.to_datetime(df['timestamp'], unit='s').dt.to_period('M')
    df['seen_at_day'] = pd.to_datetime(df['seen_at'], unit='s').dt.to_period('D')
    df = df.sort_values(by=['seen_at_day', 'timestamp_month'], ascending=True)
    df.to_csv('relay_synchronization.csv', index=False)

In [ ]:
relay_synchronization = pd.read_csv('relay_synchronization.csv')
# Copia temporanea
tmp = relay_synchronization.copy()

tmp['timestamp_month_dt'] = tmp['timestamp_month']
tmp['seen_at_day_dt'] = tmp['seen_at_day']

# Calcola frequenze
tmp = (
    tmp.groupby(['timestamp_month_dt', 'seen_at_day_dt'])
    .size()
    .reset_index(name='frequency')
)

# Pivot
heatmap_data = tmp.pivot(index='timestamp_month_dt', columns='seen_at_day_dt', values='frequency')

# Ordina l’asse Y dal più recente al meno recente
heatmap_data = heatmap_data.sort_index(ascending=False)

# Plot
fig, ax = plt.subplots(figsize=(14, 8))
sns.heatmap(
    heatmap_data,
    cmap='viridis',
    annot=True,
    fmt='g',
    linewidths=0.5,
    linecolor='gray',
    cbar_kws={'label': 'Frequency'},
    ax=ax
)

# Format x-axis (daily)
ax.set_xlabel("Seen At Day")
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')

# Format y-axis (monthly)
ax.set_ylabel("Timestamp Month")
ax.set_yticklabels(ax.get_yticklabels(), rotation=0)

# Titolo
ax.set_title("Heatmap of Seen At Day vs Timestamp Month (Frequency)")

plt.tight_layout()
plt.show()

In [ ]:
if 'relay_event_counts.csv' not in os.listdir('.'):
    query = """
    SELECT 
        relay_url,
        COUNT(*) AS event_count
    FROM 
        events_relays
    GROUP BY 
        relay_url
    ORDER BY 
        event_count ASC;
    """
    df = pd.read_sql(query, bigbrotr.conn)
    df.to_csv("relay_event_counts.csv", index=False)

relay_event_counts = pd.read_csv('relay_event_counts.csv')
relay_event_counts = pd.merge(
    relay_event_counts,
    relay_metadata.filter(['relay_url', 'network']).drop_duplicates(),
    on='relay_url'
)

In [ ]:
# Una colonna, tante righe quanti sono i network
fig, axes = plt.subplots(len(networks), 1, figsize=(10, 5 * len(networks)), sharex=False)
if len(networks) == 1:
    axes = [axes]  # Garantisce che axes sia sempre iterabile

# Genera un plot per ogni network
for i, network in enumerate(networks):
    ax = axes[i]
    if network == 'all':
        sub_df = relay_event_counts
    else:
        sub_df = relay_event_counts[relay_event_counts['network'] == network]
    
    # print top 3 relays with most events
    if network != 'all':
        top_relays = sub_df.nlargest(5, 'event_count')
        print(f"\nTop 3 relays with most events in {network}:")
        for _, row in top_relays.iterrows():
            print(f"- {row['relay_url']}: {row['event_count']} events")
        
    # Calcola CDF
    cdf = sub_df['event_count'].value_counts().sort_index().cumsum() / sub_df['event_count'].count()
    
    ax.plot(cdf.index, cdf.values, marker='o')
    ax.set_title(f'CDF of Relays vs Number of Events — Network: {network}')
    ax.set_xlabel('Number of Events')
    ax.set_ylabel('Cumulative Fraction of Relays')
    ax.set_xscale('log')
    ax.grid(True)

plt.tight_layout()
plt.show()


In [ ]:
relay_data = pd.merge(
    relay_metadata.filter(['relay_url', 'network']).drop_duplicates(),
    relay_synchronization,
    on='relay_url'
)
relay_data = pd.merge(
    relay_data,
    relay_event_counts.drop(columns='network'),
    on='relay_url'
)

In [ ]:
relay_data[relay_data['relay_url'].str.contains('oxt')]

In [ ]:
relay_data[relay_data['relay_url'].str.contains('sov')]